# Muscle Group Classification

In [1]:
from database import *
from sklearn import feature_extraction
from sklearn import preprocessing
import numpy as np

%load_ext autoreload
%autoreload 2

# Get Data from DB

In [16]:
muscle_groups = []
labels = []

# Get all exercises in DB
exercises = exercise_collection.find({})

# Get inputs and labels
for exercise in exercises:
    if str(exercise['muscle']).strip():
        muscle_groups.append(str(exercise['exercise']))
        labels.append(str(exercise['muscle']).strip())

# Formulate feature vectors

In [17]:
from sklearn.pipeline import FeatureUnion

le = preprocessing.LabelEncoder()

char_vzr = feature_extraction.text.CountVectorizer(lowercase=True, ngram_range=(3,8), analyzer='char', encoding='utf-8')
word_vzr = feature_extraction.text.CountVectorizer(lowercase=True, ngram_range=(1,5), analyzer='word', encoding='utf-8')

vectorizer = FeatureUnion([('char',char_vzr),('word',word_vzr)])

X = vectorizer.fit_transform(muscle_groups).toarray()
Y = le.fit_transform(labels)

# Inference

In [18]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
gnb = GaussianNB()

gnb.fit(X, Y)
y = gnb.predict(X)
print "Gaussian Accuracy: %f " % (sum(y == Y)/float(len(Y)))

bnb.fit(X,Y)
y = bnb.predict(X)
print "Bernoulli Accuracy: %f " % (sum(y == Y)/float(len(Y)))

Gaussian Accuracy: 0.984458 
Bernoulli Accuracy: 0.800631 


In [44]:
le.classes_[gnb.predict(vectorizer.transform(['']).toarray())]

array(['Abductors'], 
      dtype='|S11')

# Pickle Classifier

In [45]:
import pickle

pickle.dump(gnb, open('muscle_classifier.p','wb'))
pickle.dump(vectorizer, open('muscle_classifier_vectorizer.p','wb'))
pickle.dump(le, open('muscle_classifier_le.p','wb'))

In [47]:
import sys
sys.path.append('modules/')

from muscle_classifier import MuscleClassifier

mc = MuscleClassifier()
print mc.predict(['bench press'])

Counter({'chest': 1})
